In [39]:
import pandas as pd
import numpy as np
import os




In [40]:
folder_path = '../../Data/sclc_ucologne_2015/unused_data'
file_name = 'data_mrna_seq_fpkm.txt'
file_path = folder_path + '/' + file_name
# Load the RNAseq data from the csv file
data = pd.read_csv(file_path, sep='\t', index_col=0)

# Get the column names
data.head()
# the columns correspond to the patient IDs

,Entrez_Gene_Id,sclc_ucologne_2015_S00022,sclc_ucologne_2015_S00035,sclc_ucologne_2015_S00050,sclc_ucologne_2015_S00213,sclc_ucologne_2015_S00356,sclc_ucologne_2015_S00472,sclc_ucologne_2015_S00501,sclc_ucologne_2015_S00825,sclc_ucologne_2015_S00827,...,sclc_ucologne_2015_S02351,sclc_ucologne_2015_S02352,sclc_ucologne_2015_S02353,sclc_ucologne_2015_S02354,sclc_ucologne_2015_S02360,sclc_ucologne_2015_S02375,sclc_ucologne_2015_S02376,sclc_ucologne_2015_S02378,sclc_ucologne_2015_S02382,sclc_ucologne_2015_S02397
Hugo_Symbol,,,,,,,,,,,,,,,,,,,,,
A1BG,NaN,14.082400,5.465650,3.700240,5.692520,4.90083,6.65977,4.99368,6.22679,4.151240,...,5.60387,5.335420,9.839290,6.435610,6.09864,1.86425,1.324760,2.925930,1.770780,1.939300
A1CF,NaN,0.010387,0.005099,0.002786,0.001990,0.00000,0.00000,0.00000,0.00000,0.002411,...,0.00000,0.010367,0.008859,0.026770,0.00000,0.00000,0.012848,0.010893,0.004062,0.020594
A1CF,NaN,0.000402,0.000422,0.000231,0.000331,0.00000,0.00000,0.00000,0.00000,0.000200,...,0.00000,0.000395,0.000481,0.000502,0.00000,0.00000,0.000394,0.000417,0.000335,0.000395
A1CF,NaN,0.000000,0.000000,0.000000,0.000002,0.00000,0.00000,0.00000,0.00000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.019422
A1CF,NaN,0.000000,0.000000,0.000000,0.000002,0.00000,0.00000,0.00000,0.00000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000


In [41]:
# First, we'll filter out genes with zero expression across all patient samples

df_filtered = data.loc[~(data.iloc[:, 2:].eq(0).all(axis=1))]



# Next, resolve duplicates by averaging the FPKM values across duplicates

# Group by 'Hugo_Symbol', take the mean of each group, and reset the index

df_filtered = df_filtered.groupby('Hugo_Symbol').mean().reset_index()



# Display the shape of the original and the filtered DataFrame

original_shape = data.shape

filtered_shape = df_filtered.shape



original_shape, filtered_shape


((31640, 82), (18692, 83))

In [42]:
# Calculate the sum of FPKM for each sample (each column starting from the third column)
sum_fpkm_per_sample = df_filtered.iloc[:, 2:].sum()

# Convert FPKM to TPM
# Divide each FPKM value by the sum of FPKM for that sample, then multiply by 10^6
df_tpm = df_filtered.copy()
df_tpm.iloc[:, 2:] = (df_filtered.iloc[:, 2:].div(sum_fpkm_per_sample)) * 10**6

df_tpm.head()

,Hugo_Symbol,Entrez_Gene_Id,sclc_ucologne_2015_S00022,sclc_ucologne_2015_S00035,sclc_ucologne_2015_S00050,sclc_ucologne_2015_S00213,sclc_ucologne_2015_S00356,sclc_ucologne_2015_S00472,sclc_ucologne_2015_S00501,sclc_ucologne_2015_S00825,...,sclc_ucologne_2015_S02351,sclc_ucologne_2015_S02352,sclc_ucologne_2015_S02353,sclc_ucologne_2015_S02354,sclc_ucologne_2015_S02360,sclc_ucologne_2015_S02375,sclc_ucologne_2015_S02376,sclc_ucologne_2015_S02378,sclc_ucologne_2015_S02382,sclc_ucologne_2015_S02397
0,A1BG,NaN,49.841299,15.273086,14.565863,17.514792,17.393426,25.719279,16.251096,19.708647,...,22.752054,24.284115,57.793733,30.590474,20.987337,4.758543,3.586691,10.495289,7.681130,7.475808
1,A1CF,NaN,0.009546,0.003857,0.002969,0.001788,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.012246,0.013715,0.032408,0.000000,0.000000,0.008963,0.010142,0.004768,0.038945
2,A2LD1,NaN,2.319991,1.953781,27.163376,1.837371,2.598952,4.515009,5.277865,4.604362,...,9.079738,5.421795,6.572453,4.581608,11.848325,2.395508,4.724555,2.982294,12.265743,4.752917
3,A2M,NaN,99.542444,110.845385,179.958226,265.717774,446.910492,201.975527,334.034463,885.358605,...,133.800935,359.908678,282.707520,272.904387,105.083973,146.639685,110.043109,590.766883,318.268637,299.248969
4,A2ML1,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.029810,0.000000,0.000000,...,0.000000,0.078313,0.000000,0.000000,0.000000,0.000000,0.000000,0.036759,0.000000,0.000000


In [43]:
# Drop the 'Entrez_Gene_Id' column
df_tpm = df_tpm.drop('Entrez_Gene_Id', axis=1)
#change name of column Hugo Symbol to gene gene_id
#df_tpm = df_tpm.rename(columns={'Hugo_Symbol': 'gene_id'})
df_tpm = df_tpm.rename(columns={'Hugo_Symbol': 'PATIENT_ID'})

In [44]:
# transpose the dataframe
df_tpm = df_tpm.T
df_tpm.head()

,0,1,2,3,4,5,6,7,8,9,...,18682,18683,18684,18685,18686,18687,18688,18689,18690,18691
PATIENT_ID,A1BG,A1CF,A2LD1,A2M,A2ML1,A4GALT,A4GNT,AAAS,AACS,AADAC,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
sclc_ucologne_2015_S00022,49.841299,0.009546,2.319991,99.542444,0.0,22.210035,0.107257,46.724696,319.758849,0.0,...,29.22103,130.85972,5.270138,9.826973,46.252081,2.238622,32.125702,14.753678,22.122403,73.809888
sclc_ucologne_2015_S00035,15.273086,0.003857,1.953781,110.845385,0.0,27.477135,0.088028,25.159499,99.545763,0.245483,...,30.892954,208.697196,5.149757,11.938863,12.83583,3.255952,19.19287,39.743821,6.103813,36.550729
sclc_ucologne_2015_S00050,14.565863,0.002969,27.163376,179.958226,0.0,15.833247,0.033027,29.036401,115.091591,0.0,...,70.625289,82.100384,6.213945,13.321429,35.216536,3.642674,42.220943,37.605044,17.160505,37.411764
sclc_ucologne_2015_S00213,17.514792,0.001788,1.837371,265.717774,0.0,15.509113,0.040869,21.387291,35.80051,0.547173,...,26.895888,71.968483,8.997664,24.542807,15.999187,1.208275,21.599901,36.54079,11.072079,47.033629


In [45]:
# Make the gene_id row the column names
df_tpm.columns = df_tpm.iloc[0]
#delete the gene_id row
df_tpm = df_tpm.drop('PATIENT_ID')

print(df_tpm.columns)
# change "gene_id" to "PATIENT_ID"
#df_tpm.rename(columns={'gene_id': 'PATIENT_ID'}, inplace=True)
df_tpm.head()

Index(['A1BG', 'A1CF', 'A2LD1', 'A2M', 'A2ML1', 'A4GALT', 'A4GNT', 'AAAS',
       'AACS', 'AADAC',
       ...
       'ZWILCH', 'ZWINT', 'ZXDA', 'ZXDB', 'ZXDC', 'ZYG11A', 'ZYG11B', 'ZYX',
       'ZZEF1', 'ZZZ3'],
      dtype='object', name='PATIENT_ID', length=18692)


PATIENT_ID,A1BG,A1CF,A2LD1,A2M,A2ML1,A4GALT,A4GNT,AAAS,AACS,AADAC,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
sclc_ucologne_2015_S00022,49.841299,0.009546,2.319991,99.542444,0.0,22.210035,0.107257,46.724696,319.758849,0.0,...,29.22103,130.85972,5.270138,9.826973,46.252081,2.238622,32.125702,14.753678,22.122403,73.809888
sclc_ucologne_2015_S00035,15.273086,0.003857,1.953781,110.845385,0.0,27.477135,0.088028,25.159499,99.545763,0.245483,...,30.892954,208.697196,5.149757,11.938863,12.83583,3.255952,19.19287,39.743821,6.103813,36.550729
sclc_ucologne_2015_S00050,14.565863,0.002969,27.163376,179.958226,0.0,15.833247,0.033027,29.036401,115.091591,0.0,...,70.625289,82.100384,6.213945,13.321429,35.216536,3.642674,42.220943,37.605044,17.160505,37.411764
sclc_ucologne_2015_S00213,17.514792,0.001788,1.837371,265.717774,0.0,15.509113,0.040869,21.387291,35.80051,0.547173,...,26.895888,71.968483,8.997664,24.542807,15.999187,1.208275,21.599901,36.54079,11.072079,47.033629
sclc_ucologne_2015_S00356,17.393426,0.0,2.598952,446.910492,0.0,7.808751,0.0,49.961838,45.578319,4.338144,...,75.27487,102.291016,7.584379,13.0649,24.19211,1.89158,26.275205,63.737706,16.234581,38.681041


In [46]:
#save the TPM data to a csv file
#df_tpm.to_csv('../../Data/sclc_ucologne_2015/data_mrna_seq_tpm.csv', index=True)

In [47]:
BRCA= ['ESR1', 'EFTUD2', 'HSPA8', 'STAU1', 'SHMT2', 'ACTB', 'GSK3B', 'YWHAB', 'UBXN6', 'PRKRA', 'BTRC', 'DDX23', 'SSR1', 'TUBA1C', 'SNIP1', 'SRSF5', 'ERBB2', 'MKI67', 'PGR', 'PLAU']
LUAD= ['HNRNPU', 'STAU1', 'KDM1A', 'SERBP1', 'DHX9', 'EMC1', 'SSR1', 'PUM1', 'CLTC', 'PRKRA', 'KRR1', 'OCIAD1', 'CDC73', 'SLC2A1', 'HIF1A', 'PKM', 'CADM1', 'EPCAM', 'ALCAM', 'PTK7']
COAD= ['HNRNPL', 'HNRNPU', 'HNRNPA1', 'ZBTB2', 'SERBP1', 'RPL4', 'HNRNPK', 'HNRNPR', 'TFCP2', 'DHX9', 'RNF4', 'PUM1', 'ABCC1', 'CD44', 'ALCAM', 'ABCG2', 'ALDH1A1', 'ABCB1', 'EPCAM', 'PROM1']

In [48]:
# check the intersection of the genes in the BRCA, LUAD, COAD lists with the genes in the TPM dataframe
print("BRCA")
print(set(BRCA).intersection(df_tpm.columns), len(set(BRCA).intersection(df_tpm.columns)))
print("LUAD")
print(set(LUAD).intersection(df_tpm.columns),   len(set(LUAD).intersection(df_tpm.columns)))

print("COAD")
print(set(COAD).intersection(df_tpm.columns),   len(set(COAD).intersection(df_tpm.columns)))

#all three
print("all three")
print(set(BRCA).intersection(LUAD).intersection(COAD).intersection(df_tpm.columns))

BRCA
{'PLAU', 'ESR1', 'TUBA1C', 'EFTUD2', 'SNIP1', 'ERBB2', 'YWHAB', 'GSK3B', 'PRKRA', 'STAU1', 'SHMT2', 'DDX23', 'BTRC', 'UBXN6', 'ACTB', 'SSR1', 'MKI67', 'PGR', 'HSPA8', 'SRSF5'} 20
LUAD
{'SLC2A1', 'HIF1A', 'ALCAM', 'KDM1A', 'CADM1', 'OCIAD1', 'EPCAM', 'CDC73', 'PRKRA', 'DHX9', 'HNRNPU', 'PTK7', 'STAU1', 'SSR1', 'KRR1', 'SERBP1', 'PUM1', 'CLTC'} 18
COAD
{'ABCG2', 'RNF4', 'HNRNPL', 'ZBTB2', 'CD44', 'HNRNPA1', 'PUM1', 'SERBP1', 'ABCB1', 'TFCP2', 'EPCAM', 'PROM1', 'HNRNPU', 'ALDH1A1', 'HNRNPR', 'ABCC1', 'ALCAM', 'RPL4', 'DHX9', 'HNRNPK'} 20
all three
set()


In [49]:
#selected_genes =  ['TP53', 'RB1', 'TTN', 'RYR2', 'LRP1B', 'MUC16', 'ZFHX4', 'USH2A', 'CSMD3', 'NAV3', 'PCDH15', 'COL11A1', 'CSMD1', 'SYNE1', 'EYS', 'MUC17', 'ANKRD30B','FAM135B', 'FSIP2', 'TMEM132D']
selected_genes = list(set(BRCA).intersection(df_tpm.columns))
print(len(selected_genes))

df_tpm = df_tpm[selected_genes]
df_tpm.head()

20


PATIENT_ID,PLAU,ESR1,TUBA1C,EFTUD2,SNIP1,ERBB2,YWHAB,GSK3B,PRKRA,STAU1,SHMT2,DDX23,BTRC,UBXN6,ACTB,SSR1,MKI67,PGR,HSPA8,SRSF5
sclc_ucologne_2015_S00022,8.962641,0.038843,603.33087,39.486255,35.537373,5.832236,152.583767,45.028859,27.279833,26.791931,52.406212,210.116059,6.550448,92.949154,6144.547695,51.579431,69.332899,0.000004,779.794923,213.209197
sclc_ucologne_2015_S00035,29.941256,0.025724,773.790892,47.379326,12.73853,7.032399,152.561239,32.272454,68.031899,25.106981,37.191695,132.958406,5.800805,69.294119,3810.803477,55.264391,24.58061,0.000003,800.252678,71.478427
sclc_ucologne_2015_S00050,8.158813,0.201726,223.388844,63.980159,14.429779,11.648345,172.505121,46.209171,43.901371,46.550636,31.764715,123.867151,5.016077,155.896193,10179.69694,31.539071,49.345983,2.094817,888.555833,168.534547
sclc_ucologne_2015_S00213,34.59785,0.175935,493.556962,42.695883,15.345243,3.048951,124.500907,27.307735,44.287998,19.337351,31.634203,106.81509,4.336515,105.643441,7525.996002,90.578773,14.869568,0.0,846.354562,107.286149
sclc_ucologne_2015_S00356,7.768109,0.195663,665.398804,40.548981,18.329211,13.570217,214.777397,39.669672,46.379505,24.631336,70.70629,239.102064,9.611754,89.887611,7522.766737,54.078004,57.97986,0.183991,918.532326,189.903864


In [50]:
# save small subset of genes to a csv file
df_tpm.to_csv('../../Data/sclc_ucologne_2015/data_mrna_seq_tpm_small.csv', index=True)

In [51]:
# smaller data set for testing
#df_tpm.iloc[:1000, :].to_csv('../../Data/sclc_ucologne_2015/data_mrna_seq_tpm_small.tsv', index=False)


In [52]:
# merge with clinical data
clinical_data = pd.read_csv('/home/zow/prognosis-prediction/Data/sclc_ucologne_2015/data_clinical_patient.csv', sep='\t', index_col=0)
clinical_data.head()

""
"PATIENT_ID,gender,ethnicity,race,age_at_diagnosis,year_of_diagnosis,year_of_birth,overall_survival,vital_status,disease_specific_survival,primary_site"
"sclc_ucologne_2015_S00022,Male,0.0,0.0,47.0,2015,1968.0,38.0,1,38.0,Bronchus and lung"
"sclc_ucologne_2015_S00035,Female,0.0,0.0,65.0,2015,1950.0,12.0,1,12.0,Bronchus and lung"
"sclc_ucologne_2015_S00050,Male,0.0,0.0,47.0,2015,1968.0,42.0,1,42.0,Bronchus and lung"
"sclc_ucologne_2015_S00213,Male,0.0,0.0,65.0,2015,1950.0,13.0,1,13.0,Bronchus and lung"
"sclc_ucologne_2015_S00356,Female,0.0,0.0,54.0,2015,1961.0,33.0,1,33.0,Bronchus and lung"


In [53]:
merged_data = pd.merge(df_tpm, clinical_data, left_index=True, right_index=True)
merged_data.head()

PATIENT_ID,PLAU,ESR1,TUBA1C,EFTUD2,SNIP1,ERBB2,YWHAB,GSK3B,PRKRA,STAU1,SHMT2,DDX23,BTRC,UBXN6,ACTB,SSR1,MKI67,PGR,HSPA8,SRSF5
